# Student Database (MongoDB)

In [ ]:
import json
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["Students_DB"]
coll = db["students_scores"]

with open("students.json") as f:
    data = [json.loads(i) for i in f]
    
coll.insert_many(data)

In [ ]:
for x in coll.find():
    print(x)

# Student*Maximum scores in all (exam, quiz and homework)

In [ ]:
maxi=coll.aggregate([
    {"$unwind": "$scores"},
    {"$group":{"_id":"$_id","Name":{"$first":"$name"},"Type":{"$push":"$scores.type"},
              "Scores":{"$push":"$scores.score"},"Maximum Score":{"$sum":"$scores.score"}}},
    {"$sort":{"Maximum Score":-1}},{"$limit":1}
])

for i in maxi:
    print("\n Student who scored maximum scores is :",i["Name"],"\n",i)

# Students* Scored below average in Exam

In [ ]:
avge=coll.aggregate([
    {"$unwind":"$scores"},
    {"$match":{"scores.type":"exam"}},
    {"$group":{"_id":"$scores.type","Avg":{"$avg":"$scores.score"}}}
])

for i in avge:
    print(i)
    
stud=coll.aggregate([
    {"$unwind":"$scores"},
    {"$match":{"scores.type":"exam","scores.score":{"$gt":40,"$lt":i["Avg"]}}}
])

for k in stud:
    print(k)

# Students* Pass and Fail

In [ ]:
print("\n *Student who scored Below pass mark*")
fail=coll.aggregate([
    {"$unwind":"$scores"},
    {"$match":{"scores.score":{"$lt":40}}},
    {"$group":{"_id":"$_id","Name":{"$first":"$name"},"Types":{"$sum":1},"Remarks":{"$first":"Fail"}}},
    {"$sort":{"_id":1}}
])
for i in fail:
    print(i)

    
print("\n *Student who scored Above pass mark*")
pas=coll.aggregate([
    {"$unwind":"$scores"},
    {"$match":{"scores.score":{"$gte":40}}},
    {"$group":{"_id":"$_id","Name":{"$first":"$name"},"Types":{"$sum":1},"Remarks":{"$first":"Pass"}}},
    {"$sort":{"_id":1}}
])
for i in pas:
    if i['Types']==3:
        print(i)

# Students* Total, Avg and Separate collection

In [ ]:
avgtotal = db.avgtotal_coll

at = coll.aggregate([
    {"$unwind":"$scores"},
    {"$group":{"_id":"$_id", "Name":{"$first":"$name"},"Total":{"$sum":"$scores.score"},"Average":{"$avg":"$scores.score"}}},
    {"$sort":{"_id":1}}
])

j=[]
for i in at:
    j.append(i)
    print(i)
avgtotal.insert_many(j)

# Students* Scored below average and ^40% in all the categories

In [ ]:
#*Avg': 48.7*#
avgpass = db.avgpass_coll

avp = coll.aggregate([
    {"$match":{"$expr":{"$and":[{"$gt":[{"$min":"$scores.score"},40]},{"$lt":[{"$max":"$scores.score"},48.7]}]}}}
])

l=[]
for i in avp:
    l.append(i)
    print(i)
#avgpass.insert_many(l)
print("\n No students who scored below average and above 40% in all the categories***")

# Students* Scored fail mark in all the categories¶

In [ ]:
failstud = db.failstud.coll

faist = coll.aggregate([
    {"$match":{"$expr":{"$lt":[{"$max":"$scores.score"},40]}}}
])

f =[]
for i in faist:
    f.append(i)
    print(i)
failstud.insert_many(f)

# Students* Scored pass mark in all the categories

In [ ]:
passtud = db.passtud.coll

pastu = coll.aggregate([
    {"$match":{"$expr":{"$gt":[{"$min":"$scores.score"},40]}}}
])

ps = []
for i in pastu:
    ps.append(i)
    print(i)
passtud.insert_many(ps)